# 原始GAN（CGAN） mnist手写数据集生成

## 1.准备阶段

- 导库
- 定义变量
- 载入数据

## 2.模型阶段
- 生成器模型
- 判别器模型
- 定义GAN 模型（主要是固定判别器）

## 3.模型实例化
- 生成器实例化
- 判别器实例化
- GAN实例化

## 3 定义训练函数
- 计算损失，输出当前损失，输出当前模型生成图片
- 模型参数计算

## 4 main函数
- 输出结果

# 导入所需得各种库

In [1]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,optimizers,losses
from tensorflow.keras.callbacks import EarlyStopping
from IPython.display import Image
import cv2
import PIL
import json, os

# 定义会用到得全局变量

In [13]:
img_shape = (28, 28, 1)
batch_size = 200
latent_dim = 100
sample_interval=1
epochs=30

# 定义生成器

In [3]:
def build_generator():

        input=keras.Input(shape=latent_dim)

        x=layers.Dense(256)(input)
        x=layers.LeakyReLU(alpha=0.2)(x)
        x=layers.BatchNormalization(momentum=0.8)(x)

        x = layers.Dense(512)(x)
        x = layers.LeakyReLU(alpha=0.2)(x)
        x = layers.BatchNormalization(momentum=0.8)(x)

        x = layers.Dense(1024)(x)
        x = layers.LeakyReLU(alpha=0.2)(x)
        x = layers.BatchNormalization(momentum=0.8)(x)

        x=layers.Dense(np.prod(img_shape),activation='sigmoid')(x)
        output=layers.Reshape(img_shape)(x)

        model=keras.Model(inputs=input,outputs=output,name='generator')
        model.summary()
        return model

# 定义判别器

In [ ]:

def build_discriminator():
        input=keras.Input(shape=img_shape)
        x=layers.Flatten(input_shape=img_shape)(input)
        x=layers.Dense(512)(x)
        x=layers.LeakyReLU(alpha=0.2)(x)
        x=layers.Dense(256)(x)
        x=layers.LeakyReLU(alpha=0.2)(x)
        output=layers.Dense(1,activation='sigmoid')(x)
        model=keras.Model(inputs=input,outputs=output,name='discriminator')
        model.summary()
        return model

# 判别器结构 模型实例化

In [4]:
generator_model = build_generator()
discriminator_model = build_discriminator()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense (Dense)                (None, 256)               25856     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               20

# 建立GAN模型

In [ ]:
def bulid_model():
      discriminator_model.compile(loss='binary_crossentropy',
                    optimizer=keras.optimizers.Adam(0.0001, 0.000001),
                    metrics=['accuracy'])
      
      #使判别器不训练，固定判别器
      discriminator_model.trainable = False
      inputs = keras.Input(shape=latent_dim)
      img = generator_model(inputs)
      outputs = discriminator_model(img)
      model = keras.Model(inputs=inputs, outputs=outputs)
      model.summary()
      # 训练GAN
      model.compile(optimizer=keras.optimizers.Adam(0.0001, 0.000001),
                    loss='binary_crossentropy',
                    )
      return model

# GAN模型实例化

In [ ]:
model =bulid_model()

# 数据加载函数

In [ ]:
def load_data():
        (train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()
        train_images = train_images /255
        train_images = np.expand_dims(train_images, axis=3)
        print('img_number:',train_images.shape)
        return train_images

# 生成图片函数

In [6]:
def generate_sample_images(epoch):  

      row, col = 5, 5#
      noise = np.random.normal(0, 1, (row * col, 100))
      #生成噪声
      gan_imgs = generator_model.predict(noise)
      fig, axs = plt.subplots(row, col)

# 定义训练函数

In [9]:
def train():
    train_images=load_data()

    #生成标签
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    step=int(train_images.shape[0]/batch_size)
    #计算步长
    print('step:',step)

    for epoch in range(epochs):
        print(epoch)
        train_images = (tf.random.shuffle(train_images)).numpy()
        #每个epoch打乱一次
        if epoch % 2 == 0:
          # 图片展示
            generate_sample_images(epoch)

        for i in range(step):

            idx = np.arange(i*batch_size,i*batch_size+batch_size,1)
            #生成索引
            imgs =train_images[idx]
            #读取索引对应的图片
            noise = np.random.normal(0, 1, (batch_size, 100))
            gan_imgs = generator_model.predict(noise)
            #通过噪声生成图片
            #训练判别器
            discriminator_loss_real = discriminator_model.train_on_batch(imgs, valid)  # 真实数据对应标签1
            discriminator_loss_fake = discriminator_model.train_on_batch(gan_imgs, fake)  # 生成的数据对应标签0
            discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)
            #训练生成器
            # noise = np.random.normal(0, 1, (batch_size, 100))
            noise = np.random.normal(0, 1, (1000, 100))
            # 判别器训练一回，生成器生成5回
            for j in range(5):
              a=j*200
              b=(j+1)*200
              noise_temp=noise[a:b]
              generator_loss = model.train_on_batch(noise_temp, valid)
            if i%100==0:#每100步进行输出
                print("epoch:%d step:%d [discriminator_loss: %f, acc: %.2f%%] [generator_loss: %f]" % (
                    epoch,i,discriminator_loss[0], 100 * discriminator_loss[1], generator_loss))

# 定义主函数

In [ ]:
if __name__ == '__main__':
   train()